# 第一次课程作业——摄像机标定

## 任务  

- 通过3D摄像机标定物的单张图像进行摄像机标定
- 以 imgages 文件夹中的 **Jietu20200301-091513.jpg** 图片为例，该示例中选择了三个平面的交线作为世界坐标系的 x、y 和 z 轴，其中共选择了 12 个点（三个四边形的顶点）。
- 通过选定的点进行相机标定，算出 M、K、R、T。
- 验证结果，从世界坐标系中引入一些点，并检查它们是否可以通过 M 映射到相应的像素。

<div align=center><img src="images/jietu20200301-091513.jpg" width=500></div> 

## 示例代码

In [21]:
class SingleCamera:

    def __init__(self, world_coor, pixel_coor, n):

        self.__world_coor = world_coor
        self.__pixel_coor = pixel_coor
        self.__point_num = n

        '''
        0. P is the appropriate form when Pm=0
        1. SVD-solved M is known up to scale, 
        which means that the true values of the camera matrix are some scalar multiple of M,
        recorded as __roM
        2. __M can be represented as form [A b], where A is a 3x3 matrix and b is with shape 3x1
        3. __K is the intrisic Camera Matrix  
        4. __R and __t for rotation and translation
        
        '''
        self.__P = np.empty([self.__point_num, 12], dtype=float)
        self.__roM = np.empty([3, 4], dtype=float)
        self.__A = np.empty([3, 3], dtype=float)
        self.__b = np.empty([3, 1], dtype=float)
        self.__K = np.empty([3, 3], dtype=float)
        self.__R = np.empty([3, 3], dtype=float)
        self.__t = np.empty([3, 1], dtype=float)

    def returnAb(self):
        return self.__A, self.__b

    def returnKRT(self):
        return self.__K, self.__R, self.__t

    def returnM(self):
        return self.__roM

    def myReadFile(filePath):
        pass

    def changeHomo(no_homo):
        pass

    # to compose P in right form s.t. we can get Pm=0
    def composeP(self):
        i = 0
        P = np.empty([self.__point_num, 12], dtype=float)
        # print(P.shape)
        while i < self.__point_num:
            c = i // 2
            p1 = self.__world_coor[c]
            p2 = np.array([0, 0, 0, 0])
            if i % 2 == 0:
                p3 = -p1 * self.__pixel_coor[c][0]
                #print(p3)
                P[i] = np.hstack((p1, p2, p3))

            elif i % 2 == 1:
                p3 = -p1 * self.__pixel_coor[c][1]
                #print(p3)
                P[i] = np.hstack((p2, p1, p3))
            # M = P[i]
            # print(M)
            i = i + 1
        print("Now P is with form of :")
        print(P)
        print('\n')
        self.__P = P

    # svd to P，return A,b, where M=[A b]
    def svdP(self):
        U, sigma, VT = LA.svd(self.__P)
        # print(VT.shape)
        V = np.transpose(VT)
        preM = V[:, -1]
        roM = preM.reshape(3, 4)
        print("some scalar multiple of M,recorded as roM:")
        print(roM)
        print('\n')
        A = roM[0:3, 0:3].copy()
        b = roM[0:3, 3:4].copy()
        print("M can be written in form of [A b], where A is 3x3 and b is 3x1, as following:")
        print(A)
        print(b)
        print('\n')
        self.__roM = roM
        self.__A = A
        self.__b = b

    # solve the intrinsics and extrisics
    def workInAndOut(self):
        # compute ro, where ro=1/|a3|, ro may be positive or negative,
        # we choose the positive ro and name it ro01
        a3T = self.__A[2]
        # print(a3T)
        under = LA.norm(a3T)
        # print(under)
        ro01 = 1 / under
        print("The ro is %f \n" % ro01)

        # comput cx and cy
        a1T = self.__A[0]
        a2T = self.__A[1]
        cx = ro01 * ro01 * (np.dot(a1T, a3T))
        cy = ro01 * ro01 * (np.dot(a2T, a3T))
        print("cx=%f,cy=%f \n" % (cx, cy))

        # compute theta
        a_cross13 = np.cross(a1T, a3T)
        a_cross23 = np.cross(a2T, a3T)
        theta = np.arccos((-1) * np.dot(a_cross13, a_cross23) / (LA.norm(a_cross13) * LA.norm(a_cross23)))
        print("theta is: %f \n" % theta)

        # compute alpha and beta
        alpha = ro01 * ro01 * LA.norm(a_cross13) * np.sin(theta)
        beta = ro01 * ro01 * LA.norm(a_cross23) * np.sin(theta)
        print("alpha:%f, beta:%f \n" % (alpha,beta))

        # compute K
        K = np.array([alpha, -alpha * (1 / np.tan(theta)), cx, 0, beta / (np.sin(theta)), cy, 0, 0, 1])
        K = K.reshape(3, 3)
        print("We can get K accordingly: ")
        print(K)
        print('\n')
        self.__K = K

        # compute R
        r1 = a_cross23 / LA.norm(a_cross23)
        r301 = ro01 * a3T
        r2 = np.cross(r301, r1)
        #print(r1, r2, r301)
        R = np.hstack((r1, r2, r301))
        R = R.reshape(3,3)
        print("we can get R:")
        print(R)
        print('\n')
        self.__R = R

        # compute T
        T = ro01 * np.dot(LA.inv(K), self.__b)
        print("we can get t:")
        print(T)
        print('\n')
        self.__t = T

    def selfcheck(self, w_check, c_check):
        my_size = c_check.shape[0]
        my_err = np.empty([my_size])
        for i in range(my_size) :
            test_pix = np.dot(self.__roM, w_check[i])
            u = test_pix[0] / test_pix[2]
            v = test_pix[1] / test_pix[2]
            u_c = c_check[i][0]
            v_c = c_check[i][1]
            print("you get test point %d with result (%f,%f)" % (i, u, v))
            print("the correct result is (%f,%f)" % (u_c,v_c))
            my_err[i] = (abs(u-u_c)/u_c+abs(v-v_c)/v_c)/2
        average_err = my_err.sum()/my_size
        print("The average error is %f ," % average_err)
        if average_err > 0.1:
            print("which is more than 0.1")
        else:
            print("which is smaller than 0.1, the M is acceptable")

## 结果测试

- **导入相关模块**

In [22]:
import numpy as np
from numpy import linalg as LA

- **写入所有点的世界坐标，像素坐标，设置测试点**

In [23]:
# The homogeneous world coodinate

# Although it would be more appropriate to write a function to read the coordinates, 
# we've simplified it by listing the coordinates directly in array.

# world corrdinate
# points: (8, 0, 9), (8, 0, 1), (6, 0, 1), (6, 0, 9)
w_xz = np.array([8, 0, 9, 1, 8, 0, 1, 1, 6, 0, 1, 1, 6, 0, 9, 1])
w_xz = w_xz.reshape(4, 4)
# points: (5, 1, 0), (5, 9, 0), (4, 9, 0), (4, 1, 0)
w_xy = np.array([5, 1, 0, 1, 5, 9, 0, 1, 4, 9, 0, 1, 4, 1, 0, 1])
w_xy = w_xy.reshape(4, 4)
# points: (0, 4, 7), (0, 4, 3), (0, 8, 3), (0, 8, 7)
w_yz = np.array([0, 4, 7, 1, 0, 4, 3, 1, 0, 8, 3, 1, 0, 8, 7, 1])
w_yz = w_yz.reshape(4, 4)
w_coor = np.vstack((w_xz, w_xy, w_yz))
#print(w_coor)
# pixel coordinate
c_xz = np.array([275, 142, 312, 454, 382, 436, 357, 134])
c_xz = c_xz.reshape(4, 2)
c_xy = np.array([432, 473, 612, 623, 647, 606, 464, 465])
c_xy = c_xy.reshape(4, 2)
c_yz = np.array([654, 216, 644, 368, 761, 420, 781, 246])
c_yz = c_yz.reshape(4, 2)
c_coor = np.vstack((c_xz, c_xy, c_yz))
#print(c_coor)
# coordinate for validation whether the M is correct or not
w_check = np.array([6, 0, 5, 1, 3, 3, 0, 1, 0, 4, 0, 1, 0, 4, 4, 1, 0, 0, 7, 1])
w_check = w_check.reshape(5, 4)
c_check = np.array([369, 297, 531, 484, 640, 468, 646, 333, 556, 194])
c_check = c_check.reshape(5, 2)

- **计算相机参数，测试投影矩阵**

In [24]:
aCamera = SingleCamera(w_coor, c_coor, 12)  # 12 points in total are used
aCamera.composeP()
aCamera.svdP()
aCamera.workInAndOut()  # print computed result
aCamera.selfcheck(w_check,c_check)  # test 5 points and verify M

Now P is with form of :
[[ 8.000e+00  0.000e+00  9.000e+00  1.000e+00  0.000e+00  0.000e+00
   0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  8.000e+00  0.000e+00
   9.000e+00  1.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 8.000e+00  0.000e+00  1.000e+00  1.000e+00  0.000e+00  0.000e+00
   0.000e+00  0.000e+00 -2.496e+03  0.000e+00 -3.120e+02 -3.120e+02]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  8.000e+00  0.000e+00
   1.000e+00  1.000e+00 -3.632e+03  0.000e+00 -4.540e+02 -4.540e+02]
 [ 6.000e+00  0.000e+00  1.000e+00  1.000e+00  0.000e+00  0.000e+00
   0.000e+00  0.000e+00 -2.292e+03  0.000e+00 -3.820e+02 -3.820e+02]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  6.000e+00  0.000e+00
   1.000e+00  1.000e+00 -2.616e+03  0.000e+00 -4.360e+02 -4.360e+02]
 [ 6.000e+00  0.000e+00  9.000e+00  1.000e+00  0.000e+00  0.000e+00
   0.000e+00  0.000e+00 -2.142e+03  0.000e+00 -3.213e+03 -3.570e+02]
 [ 0.000e+00  0.0